#Insall Package

In [ ]:
!pip install pulp

In [ ]:
!pip install loguru

In [ ]:
!apt-get install glpk-utils

In [ ]:
!apt-get install coinor-cbc

In [ ]:
!pip install gurobipy

# Import Library

In [63]:
import json
import pulp
from loguru import logger
import itertools
import pandas as pd
df= pd.read_csv("acc_london_800_new2.csv")

# Load data from JSON file
all_data = json.loads(json.dumps(list(df.T.to_dict().values())))


In [64]:
df.head()

Unnamed: 0      origin_id  destination_id  severity  casualties  \
0           0  200701XH65182           40095         3           3   
1           1  200701XH65182           42194         3           3   
2           2  200701XH65182         2917654         3           3   
3           3  200701XH65182         3995215         3           3   
4           4  200701XH65182         4622618         3           3   

   Distance in meter  duration in seconds  duration with traffic in sec  \
0               5590                  732                           790   
1               5590                  732                           790   
2               6927                 1162                          1360   
3              12748                 1026                          1102   
4              10002                 1212                          1310   

   return duration in seconds  return duration with traffic in sec  
0                         711                                  746  
1                         711                                  746  
2                        1160                                 1156  
3                        1068                                 1177  
4                        1256                                 1337

In [65]:
len_accident = len(df.origin_id.unique().tolist())

# Optimization Solution With Traffic Time (GO + Return)

In [19]:
Sum_Cost_With_Traffic =0
Avg_Cost_With_Traffic = 0

Trafic_Go_return = {}
Trafic_Go_return_optimal = []
Trafic_Go_return_frame_optimal = pd.DataFrame()

In [20]:
for key, group in itertools.groupby(all_data, key=lambda x: x["origin_id"]):


        data = {}
        data[key] = list(group)
        data = data[key]


        # Define the problem as a minimization problem
        problem = pulp.LpProblem("Ambulance Delivery Problem", pulp.LpMinimize)

        # Create decision variables
        decision_variables = []
        for i, row in enumerate(data):
            variable = pulp.LpVariable(f"X_{i}", lowBound=0, cat='Integer')
            decision_variables.append(variable)

        # Create objective function with decision variable
        total_cost = pulp.lpSum([(decision_variables[i] * (
        int(data[i]['Distance in meter']) *2 +
        10* int(data[i]['duration with traffic in sec']) * int(data[i]['severity']) +
        10*int(data[i]['return duration with traffic in sec']) * int(data[i]['severity'])))
        for i in range(len(data))])
        problem += total_cost

        # Create constraints
        for i in range(len(data)):
            problem += (decision_variables[i] >= 0)
        problem += (pulp.lpSum([decision_variables[i] for i in range(len(data))]) == 1)

        # Solve problem with different algorithms
        solvers = [
            pulp.getSolver('PULP_CBC_CMD', timeLimit=1200),
          #  pulp.GLPK_CMD(msg=True),
           # pulp.GUROBI(),
        ]



        for solver in solvers:
            problem.solve(solver)
            objective_value = pulp.value(problem.objective)
            Sum_Cost_With_Traffic += objective_value
            Trafic_Go_return = {}
            Trafic_Go_return['cost_of_PULP_CBC_CMD'] = objective_value

            # Print optimal solution for every origin_id

            optimal_destinations = {}
            for i, row in enumerate(data):
                if decision_variables[i].value() > 0:
                    origin_id = row['origin_id']
                    destination_id = row['destination_id']
                    if origin_id not in optimal_destinations or \
                            data[optimal_destinations[origin_id]]['duration with traffic in sec'] > row['duration with traffic in sec']:
                        optimal_destinations[origin_id] = i

            optimal_solutions = {}
            for origin_id in optimal_destinations:
                optimal_solutions[origin_id] = []
                for i in range(len(data)):
                    row = data[i]
                    if row['origin_id'] == origin_id and decision_variables[i].value() > 0:
                        optimal_solutions[origin_id].append((row['destination_id'], row['Distance in meter'],row['duration with traffic in sec'],row['return duration with traffic in sec'] ,row['severity']))

            for origin_id in sorted(optimal_solutions.keys()):
               # solutions_str = "\n".join([f"{x[0]} (distance: {x[1]} meters, duration with traffic: {x[2]} sec, return duration with traffic: {x[3]} sec, severity: {x[4]})" for x in optimal_solutions[origin_id]])
                for x in optimal_solutions[origin_id]:
                   Trafic_Go_return['accident'] = origin_id
                   Trafic_Go_return['hospital'] = x[0]
                   Trafic_Go_return['distance'] = x[1]
                   Trafic_Go_return['duration with traffic'] = x[2]
                   Trafic_Go_return['return duration with traffic'] = x[3]
                   Trafic_Go_return['severity'] = x[4]
                   Trafic_Go_return_optimal.append(Trafic_Go_return)

C:\Users\wesam\anaconda3\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [21]:
print(Trafic_Go_return_optimal)

[{'cost_of_PULP_CBC_CMD': 57260.0, 'accident': '200701XH65182', 'hospital': 40095, 'distance': 5590, 'duration with traffic': 790, 'return duration with traffic': 746, 'severity': 3}, {'cost_of_PULP_CBC_CMD': 27676.0, 'accident': '200901QA10383', 'hospital': 40083, 'distance': 2033, 'duration with traffic': 355, 'return duration with traffic': 432, 'severity': 3}, {'cost_of_PULP_CBC_CMD': 37382.0, 'accident': '200501KF60564', 'hospital': 46586, 'distance': 2386, 'duration with traffic': 530, 'return duration with traffic': 557, 'severity': 3}, {'cost_of_PULP_CBC_CMD': 30442.0, 'accident': '201001RG40583', 'hospital': 462443, 'distance': 3791, 'duration with traffic': 341, 'return duration with traffic': 421, 'severity': 3}, {'cost_of_PULP_CBC_CMD': 36670.0, 'accident': '201201JC30566', 'hospital': 39964, 'distance': 2150, 'duration with traffic': 754, 'return duration with traffic': 325, 'severity': 3}, {'cost_of_PULP_CBC_CMD': 42518.0, 'accident': '201401ZD31157', 'hospital': 40924, '

In [22]:
len(Trafic_Go_return_optimal)

800

In [23]:
Trafic_Go_return_frame_optimal = pd.DataFrame.from_records(Trafic_Go_return_optimal)

In [24]:
Trafic_Go_return_frame_optimal.head(800)

cost_of_PULP_CBC_CMD       accident  hospital  distance  \
0                 57260.0  200701XH65182     40095      5590   
1                 27676.0  200901QA10383     40083      2033   
2                 37382.0  200501KF60564     46586      2386   
3                 30442.0  201001RG40583    462443      3791   
4                 36670.0  201201JC30566     39964      2150   
..                    ...            ...       ...       ...   
795               17548.0  201201QK50297     66513      1109   
796               29074.0  200601EO40267     66507      2147   
797                7672.0  201001RG40143   2704335       716   
798               20078.0  200601XD80655   2917654      2074   
799               11446.0  200801RY10207   1819387       728   

     duration with traffic  return duration with traffic  severity  
0                      790                           746         3  
1                      355                           432         3  
2                      530                           557         3  
3                      341                           421         3  
4                      754                           325         3  
..                     ...                           ...       ...  
795                    250                           261         3  
796                    471                           355         3  
797                     63                           145         3  
798                    183                           348         3  
799                    212                           121         3  

[800 rows x 7 columns]

# Optimization Solution WithOut Traffic Time (GO + Return)

In [38]:
Sum_Cost_WithOut_Traffic =0
Avg_Cost_WithOut_Traffic = 0


WithOutTrafic_Go_return = {}
WithOutTrafic_Go_return_optimal = []
WithOutTrafic_Go_return_frame_optimal  = pd.DataFrame()

In [39]:
for key, group in itertools.groupby(all_data, key=lambda x: x["origin_id"]):

        data = {}
        data[key] = list(group)
        data = data[key]


        # Define the problem as a minimization problem
        problem = pulp.LpProblem("Ambulance Delivery Problem", pulp.LpMinimize)

        # Create decision variables
        decision_variables = []
        for i, row in enumerate(data):
            variable = pulp.LpVariable(f"X_{i}", lowBound=0, cat='Integer')
            decision_variables.append(variable)

        # Create objective function with decision variable
        total_cost = pulp.lpSum([(decision_variables[i] * (
            int(data[i]['Distance in meter']) * 2 +
            10*int(data[i]['duration in seconds']) * int(data[i]['severity'])+
            10*int(data[i]['return duration in seconds']) * int(data[i]['severity'])))

        for i in range(len(data))])
        problem += total_cost

        # Create constraints
        for i in range(len(data)):
            problem += (decision_variables[i] >= 0)
        problem += (pulp.lpSum([decision_variables[i] for i in range(len(data))]) == 1)



        # Solve problem with different algorithms
        solvers = [
            pulp.getSolver('PULP_CBC_CMD', timeLimit=1200),
            #pulp.GLPK_CMD(msg=True),
            #pulp.GUROBI(),
        ]



        for solver in solvers:
            problem.solve(solver)
            objective_value = pulp.value(problem.objective)
            Sum_Cost_WithOut_Traffic += objective_value
            WithOutTrafic_Go_return = {}
            WithOutTrafic_Go_return['cost_of_PULP_CBC_CMD'] = objective_value

            # Print optimal solution for every origin_id

            optimal_destinations = {}
            for i, row in enumerate(data):
                if decision_variables[i].value() > 0:
                    origin_id = row['origin_id']
                    destination_id = row['destination_id']
                    if origin_id not in optimal_destinations or \
                            data[optimal_destinations[origin_id]]['duration in seconds'] > row['duration in seconds']:
                        optimal_destinations[origin_id] = i

            optimal_solutions = {}
            for origin_id in optimal_destinations:
                optimal_solutions[origin_id] = []
                for i in range(len(data)):
                    row = data[i]
                    if row['origin_id'] == origin_id and decision_variables[i].value() > 0:
                        optimal_solutions[origin_id].append((row['destination_id'], row['Distance in meter'],row['duration in seconds'],row['return duration in seconds'] ,row['severity']))

            for origin_id in sorted(optimal_solutions.keys()):
                 for x in optimal_solutions[origin_id]:
                   WithOutTrafic_Go_return['accident'] = origin_id
                   WithOutTrafic_Go_return['hospital'] = x[0]
                   WithOutTrafic_Go_return['distance'] = x[1]
                   WithOutTrafic_Go_return['duration without traffic'] = x[2]
                   WithOutTrafic_Go_return['return duration without traffic'] = x[3]
                   WithOutTrafic_Go_return['severity'] = x[4]
                   WithOutTrafic_Go_return_optimal.append(WithOutTrafic_Go_return)

C:\Users\wesam\anaconda3\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [40]:
print(WithOutTrafic_Go_return_optimal)

[{'cost_of_PULP_CBC_CMD': 54470.0, 'accident': '200701XH65182', 'hospital': 40095, 'distance': 5590, 'duration without traffic': 732, 'return duration without traffic': 711, 'severity': 3}, {'cost_of_PULP_CBC_CMD': 24496.0, 'accident': '200901QA10383', 'hospital': 40083, 'distance': 2033, 'duration without traffic': 337, 'return duration without traffic': 344, 'severity': 3}, {'cost_of_PULP_CBC_CMD': 33122.0, 'accident': '200501KF60564', 'hospital': 46586, 'distance': 2386, 'duration without traffic': 452, 'return duration without traffic': 493, 'severity': 3}, {'cost_of_PULP_CBC_CMD': 41422.0, 'accident': '201001RG40583', 'hospital': 462443, 'distance': 3791, 'duration without traffic': 466, 'return duration without traffic': 662, 'severity': 3}, {'cost_of_PULP_CBC_CMD': 31330.0, 'accident': '201201JC30566', 'hospital': 39964, 'distance': 2150, 'duration without traffic': 603, 'return duration without traffic': 298, 'severity': 3}, {'cost_of_PULP_CBC_CMD': 41138.0, 'accident': '201401

In [43]:
WithOutTrafic_Go_return_frame_optimal = pd.DataFrame.from_records(WithOutTrafic_Go_return_optimal)

In [44]:
WithOutTrafic_Go_return_frame_optimal.head(800)

cost_of_PULP_CBC_CMD       accident  hospital  distance  \
0                 54470.0  200701XH65182     40095      5590   
1                 24496.0  200901QA10383     40083      2033   
2                 33122.0  200501KF60564     46586      2386   
3                 41422.0  201001RG40583    462443      3791   
4                 31330.0  201201JC30566     39964      2150   
..                    ...            ...       ...       ...   
795               16408.0  201201QK50297     66513      1109   
796               33784.0  200601EO40267     66507      2147   
797                8632.0  201001RG40143   2704335       716   
798               23648.0  200601XD80655   2917654      2074   
799               13636.0  200801RY10207   1819387       728   

     duration without traffic  return duration without traffic  severity  
0                         732                              711         3  
1                         337                              344         3  
2                         452                              493         3  
3                         466                              662         3  
4                         603                              298         3  
..                        ...                              ...       ...  
795                       235                              238         3  
796                       553                              430         3  
797                        69                              171         3  
798                       218                              432         3  
799                       267                              139         3  

[800 rows x 7 columns]

# Optimization Solution With Traffic Time (GO)

In [25]:
Sum_Cost_With_Traffic_GO = 0
Avg_Cost_With_Traffic_GO = 0

Trafic_Go = {}
Trafic_Go_optimal = []
Trafic_Go_frame_optimal = pd.DataFrame()

In [26]:
for key, group in itertools.groupby(all_data, key=lambda x: x["origin_id"]):

    data = {}
    data[key] = list(group)
    data = data[key]

    # Define the problem as a minimization problem
    problem = pulp.LpProblem("Ambulance Delivery Problem", pulp.LpMinimize)

    # Create decision variables
    decision_variables = []
    for i, row in enumerate(data):
        variable = pulp.LpVariable(f"X_{i}", lowBound=0, cat='Integer')
        decision_variables.append(variable)

    # Create objective function with decision variable
    total_cost = pulp.lpSum([(decision_variables[i] * (
        int(data[i]['Distance in meter']) +
        10*int(data[i]['duration with traffic in sec']) * int(data[i]['severity'])))
        for i in range(len(data))])
    problem += total_cost

    # Create constraints
    for i in range(len(data)):
        problem += (decision_variables[i] >= 0)
    problem += (pulp.lpSum([decision_variables[i] for i in range(len(data))]) == 1)

    # Solve problem with different algorithms
    solvers = [
        pulp.getSolver('PULP_CBC_CMD', timeLimit=1200),
       # pulp.GLPK_CMD(msg=True),
       #pulp.GUROBI(),
    ]

    for solver in solvers:
        problem.solve(solver)
        objective_value = pulp.value(problem.objective)
        Sum_Cost_With_Traffic_GO += objective_value
        Trafic_Go = {}
        Trafic_Go['cost_of_PULP_CBC_CMD'] = objective_value

        # Print optimal solution for every origin_id

        optimal_destinations = {}
        for i, row in enumerate(data):
            if decision_variables[i].value() > 0:
                origin_id = row['origin_id']
                destination_id = row['destination_id']
                if origin_id not in optimal_destinations or \
                        data[optimal_destinations[origin_id]]['duration with traffic in sec'] > row[
                    'duration with traffic in sec']:
                    optimal_destinations[origin_id] = i

        optimal_solutions = {}
        for origin_id in optimal_destinations:
            optimal_solutions[origin_id] = []
            for i in range(len(data)):
                row = data[i]
                if row['origin_id'] == origin_id and decision_variables[i].value() > 0:
                    optimal_solutions[origin_id].append((row['destination_id'], row['Distance in meter'],
                                                         row['duration with traffic in sec'],row['return duration with traffic in sec'],
                                                         row['severity']))

        for origin_id in sorted(optimal_solutions.keys()):
          for x in optimal_solutions[origin_id]:
                   Trafic_Go['accident'] = origin_id
                   Trafic_Go['hospital'] = x[0]
                   Trafic_Go['distance'] = x[1]
                   Trafic_Go['duration with traffic'] = x[2]
                   Trafic_Go['return duration with traffic'] = x[3]
                   Trafic_Go['severity'] = x[4]
                   Trafic_Go_optimal.append(Trafic_Go)


C:\Users\wesam\anaconda3\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [30]:
print(Trafic_Go_optimal)

[{'cost_of_PULP_CBC_CMD': 29290.0, 'accident': '200701XH65182', 'hospital': 40095, 'distance': 5590, 'duration with traffic': 790, 'return duration with traffic': 746, 'severity': 3}, {'cost_of_PULP_CBC_CMD': 12683.0, 'accident': '200901QA10383', 'hospital': 40083, 'distance': 2033, 'duration with traffic': 355, 'return duration with traffic': 432, 'severity': 3}, {'cost_of_PULP_CBC_CMD': 18286.0, 'accident': '200501KF60564', 'hospital': 46586, 'distance': 2386, 'duration with traffic': 530, 'return duration with traffic': 557, 'severity': 3}, {'cost_of_PULP_CBC_CMD': 14021.0, 'accident': '201001RG40583', 'hospital': 462443, 'distance': 3791, 'duration with traffic': 341, 'return duration with traffic': 421, 'severity': 3}, {'cost_of_PULP_CBC_CMD': 18569.0, 'accident': '201201JC30566', 'hospital': 18395, 'distance': 5819, 'duration with traffic': 425, 'return duration with traffic': 636, 'severity': 3}, {'cost_of_PULP_CBC_CMD': 20794.0, 'accident': '201401ZD31157', 'hospital': 40924, '

In [31]:
Trafic_Go_frame_optimal = pd.DataFrame.from_records(Trafic_Go_optimal)

In [32]:
Trafic_Go_frame_optimal.head(800)

cost_of_PULP_CBC_CMD       accident  hospital  distance  \
0                 29290.0  200701XH65182     40095      5590   
1                 12683.0  200901QA10383     40083      2033   
2                 18286.0  200501KF60564     46586      2386   
3                 14021.0  201001RG40583    462443      3791   
4                 18569.0  201201JC30566     18395      5819   
..                    ...            ...       ...       ...   
795                8609.0  201201QK50297     66513      1109   
796               16277.0  200601EO40267     66507      2147   
797                2606.0  201001RG40143   2704335       716   
798                7564.0  200601XD80655   2917654      2074   
799                7088.0  200801RY10207   1819387       728   

     duration with traffic  return duration with traffic  severity  
0                      790                           746         3  
1                      355                           432         3  
2                      530                           557         3  
3                      341                           421         3  
4                      425                           636         3  
..                     ...                           ...       ...  
795                    250                           261         3  
796                    471                           355         3  
797                     63                           145         3  
798                    183                           348         3  
799                    212                           121         3  

[800 rows x 7 columns]

# ptimization Solution With Traffic Time (Return)

In [33]:
Sum_Cost_With_Traffic_Return = 0
Avg_Cost_With_Traffic_Return = 0

Trafic_Return = {}
Trafic_Return_optimal = []
Trafic_Return_frame_optimal = pd.DataFrame()

In [34]:
for key, group in itertools.groupby(all_data, key=lambda x: x["origin_id"]):

    data = {}
    data[key] = list(group)
    data = data[key]

    # Define the problem as a minimization problem
    problem = pulp.LpProblem("Ambulance Delivery Problem", pulp.LpMinimize)

    # Create decision variables
    decision_variables = []
    for i, row in enumerate(data):
        variable = pulp.LpVariable(f"X_{i}", lowBound=0, cat='Integer')
        decision_variables.append(variable)

    # Create objective function with decision variable
    total_cost = pulp.lpSum([(decision_variables[i] * (
                int(data[i]['Distance in meter']) +
        10*int(data[i]['return duration with traffic in sec']) * int(data[i]['severity'])))
        for i in range(len(data))])
    problem += total_cost

    # Create constraints
    for i in range(len(data)):
        problem += (decision_variables[i] >= 0)
    problem += (pulp.lpSum([decision_variables[i] for i in range(len(data))]) == 1)

    # Solve problem with different algorithms
    solvers = [
        pulp.getSolver('PULP_CBC_CMD', timeLimit=1200),
        #pulp.GLPK_CMD(msg=True),
        #pulp.GUROBI(),
    ]

    for solver in solvers:
        problem.solve(solver)
        objective_value = pulp.value(problem.objective)
        Sum_Cost_With_Traffic_Return +=objective_value
        Trafic_Return = {}
        Trafic_Return['cost_of_PULP_CBC_CMD'] = objective_value

        # Print optimal solution for every origin_id

        optimal_destinations = {}
        for i, row in enumerate(data):
            if decision_variables[i].value() > 0:
                origin_id = row['origin_id']
                destination_id = row['destination_id']
                if origin_id not in optimal_destinations or \
                        data[optimal_destinations[origin_id]]['return duration with traffic in sec'] > row[
                    'return duration with traffic in sec']:
                    optimal_destinations[origin_id] = i

        optimal_solutions = {}
        for origin_id in optimal_destinations:
            optimal_solutions[origin_id] = []
            for i in range(len(data)):
                row = data[i]
                if row['origin_id'] == origin_id and decision_variables[i].value() > 0:
                    optimal_solutions[origin_id].append((row['destination_id'], row['Distance in meter'],row['duration with traffic in sec'],
                                                         row['return duration with traffic in sec'], row['severity']))

        for origin_id in sorted(optimal_solutions.keys()):
            for x in optimal_solutions[origin_id]:
                   Trafic_Return['accident'] = origin_id
                   Trafic_Return['hospital'] = x[0]
                   Trafic_Return['distance'] = x[1]
                   Trafic_Return['duration with traffic'] = x[2]
                   Trafic_Return['return duration with traffic'] = x[3]
                   Trafic_Return['severity'] = x[4]
                   Trafic_Return_optimal.append(Trafic_Return)

C:\Users\wesam\anaconda3\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [35]:
print(Trafic_Return_optimal)

[{'cost_of_PULP_CBC_CMD': 27970.0, 'accident': '200701XH65182', 'hospital': 40095, 'distance': 5590, 'duration with traffic': 790, 'return duration with traffic': 746, 'severity': 3}, {'cost_of_PULP_CBC_CMD': 14993.0, 'accident': '200901QA10383', 'hospital': 40083, 'distance': 2033, 'duration with traffic': 355, 'return duration with traffic': 432, 'severity': 3}, {'cost_of_PULP_CBC_CMD': 19096.0, 'accident': '200501KF60564', 'hospital': 46586, 'distance': 2386, 'duration with traffic': 530, 'return duration with traffic': 557, 'severity': 3}, {'cost_of_PULP_CBC_CMD': 16421.0, 'accident': '201001RG40583', 'hospital': 462443, 'distance': 3791, 'duration with traffic': 341, 'return duration with traffic': 421, 'severity': 3}, {'cost_of_PULP_CBC_CMD': 11900.0, 'accident': '201201JC30566', 'hospital': 39964, 'distance': 2150, 'duration with traffic': 754, 'return duration with traffic': 325, 'severity': 3}, {'cost_of_PULP_CBC_CMD': 21724.0, 'accident': '201401ZD31157', 'hospital': 40924, '

In [36]:
Trafic_Return_frame_optimal = pd.DataFrame.from_records(Trafic_Return_optimal)

In [37]:
Trafic_Return_frame_optimal.head(800)

cost_of_PULP_CBC_CMD       accident  hospital  distance  \
0                 27970.0  200701XH65182     40095      5590   
1                 14993.0  200901QA10383     40083      2033   
2                 19096.0  200501KF60564     46586      2386   
3                 16421.0  201001RG40583    462443      3791   
4                 11900.0  201201JC30566     39964      2150   
..                    ...            ...       ...       ...   
795                8939.0  201201QK50297     66513      1109   
796               12797.0  200601EO40267     66507      2147   
797                5066.0  201001RG40143   2704335       716   
798               12514.0  200601XD80655   2917654      2074   
799                4358.0  200801RY10207   1819387       728   

     duration with traffic  return duration with traffic  severity  
0                      790                           746         3  
1                      355                           432         3  
2                      530                           557         3  
3                      341                           421         3  
4                      754                           325         3  
..                     ...                           ...       ...  
795                    250                           261         3  
796                    471                           355         3  
797                     63                           145         3  
798                    183                           348         3  
799                    212                           121         3  

[800 rows x 7 columns]

# Optimization Solution WithOut Traffic Time (GO)

In [45]:
Sum_Cost_WithOut_Traffic_GO = 0
Avg_Cost_WithOut_Traffic_GO = 0

WithOutTrafic_Go = {}
WithOutTrafic_Go_optimal = []
WithOutTrafic_Go_frame_optimal = pd.DataFrame()

In [46]:
for key, group in itertools.groupby(all_data, key=lambda x: x["origin_id"]):

    data = {}
    data[key] = list(group)
    data = data[key]

    # Define the problem as a minimization problem
    problem = pulp.LpProblem("Ambulance Delivery Problem", pulp.LpMinimize)

    # Create decision variables
    decision_variables = []
    for i, row in enumerate(data):
        variable = pulp.LpVariable(f"X_{i}", lowBound=0, cat='Integer')
        decision_variables.append(variable)

    # Create objective function with decision variable
    total_cost = pulp.lpSum([(decision_variables[i] * (
                int(data[i]['Distance in meter']) +  10*int(data[i]['duration in seconds']) * int(data[i]['severity'])))
                             for i in range(len(data))])
    problem += total_cost

    # Create constraints
    for i in range(len(data)):
        problem += (decision_variables[i] >= 0)
    problem += (pulp.lpSum([decision_variables[i] for i in range(len(data))]) == 1)

    # Solve problem with different algorithms
    solvers = [
        pulp.getSolver('PULP_CBC_CMD', timeLimit=1200),
        #pulp.GLPK_CMD(msg=True),
        #pulp.GUROBI(),
    ]

    for solver in solvers:
        problem.solve(solver)
        objective_value = pulp.value(problem.objective)
        Sum_Cost_WithOut_Traffic_GO +=objective_value
        WithOut_Trafic_Go = {}
        WithOut_Trafic_Go['cost_of_PULP_CBC_CMD'] = objective_value

        # Print optimal solution for every origin_id

        optimal_destinations = {}
        for i, row in enumerate(data):
            if decision_variables[i].value() > 0:
                origin_id = row['origin_id']
                destination_id = row['destination_id']
                if origin_id not in optimal_destinations or \
                        data[optimal_destinations[origin_id]]['duration in seconds'] > row[
                    'duration in seconds']:
                    optimal_destinations[origin_id] = i

        optimal_solutions = {}
        for origin_id in optimal_destinations:
            optimal_solutions[origin_id] = []
            for i in range(len(data)):
                row = data[i]
                if row['origin_id'] == origin_id and decision_variables[i].value() > 0:
                    optimal_solutions[origin_id].append((row['destination_id'], row['Distance in meter'],
                                                         row['duration in seconds'],row['return duration in seconds'],
                                                         row['severity']))

        for origin_id in sorted(optimal_solutions.keys()):
            for x in optimal_solutions[origin_id]:
                   WithOut_Trafic_Go['accident'] = origin_id
                   WithOut_Trafic_Go['hospital'] = x[0]
                   WithOut_Trafic_Go['distance'] = x[1]
                   WithOut_Trafic_Go['duration without traffic'] = x[2]
                   WithOut_Trafic_Go['return duration without traffic'] = x[3]
                   WithOut_Trafic_Go['severity'] = x[4]
                   WithOutTrafic_Go_optimal.append(WithOut_Trafic_Go)


C:\Users\wesam\anaconda3\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [47]:
print(WithOutTrafic_Go_optimal)

[{'cost_of_PULP_CBC_CMD': 27550.0, 'accident': '200701XH65182', 'hospital': 40095, 'distance': 5590, 'duration without traffic': 732, 'return duration without traffic': 711, 'severity': 3}, {'cost_of_PULP_CBC_CMD': 12143.0, 'accident': '200901QA10383', 'hospital': 40083, 'distance': 2033, 'duration without traffic': 337, 'return duration without traffic': 344, 'severity': 3}, {'cost_of_PULP_CBC_CMD': 15946.0, 'accident': '200501KF60564', 'hospital': 46586, 'distance': 2386, 'duration without traffic': 452, 'return duration without traffic': 493, 'severity': 3}, {'cost_of_PULP_CBC_CMD': 17771.0, 'accident': '201001RG40583', 'hospital': 462443, 'distance': 3791, 'duration without traffic': 466, 'return duration without traffic': 662, 'severity': 3}, {'cost_of_PULP_CBC_CMD': 14662.0, 'accident': '201201JC30566', 'hospital': 69632, 'distance': 3922, 'duration without traffic': 358, 'return duration without traffic': 432, 'severity': 3}, {'cost_of_PULP_CBC_CMD': 20824.0, 'accident': '201401

In [48]:
WithOutTrafic_Go_frame_optimal = pd.DataFrame.from_records(WithOutTrafic_Go_optimal)

In [49]:
WithOutTrafic_Go_frame_optimal.head(800)

cost_of_PULP_CBC_CMD       accident  hospital  distance  \
0                 27550.0  200701XH65182     40095      5590   
1                 12143.0  200901QA10383     40083      2033   
2                 15946.0  200501KF60564     46586      2386   
3                 17771.0  201001RG40583    462443      3791   
4                 14662.0  201201JC30566     69632      3922   
..                    ...            ...       ...       ...   
795                8159.0  201201QK50297     66513      1109   
796               18737.0  200601EO40267     66507      2147   
797                2786.0  201001RG40143   2704335       716   
798                8614.0  200601XD80655   2917654      2074   
799                8738.0  200801RY10207   1819387       728   

     duration without traffic  return duration without traffic  severity  
0                         732                              711         3  
1                         337                              344         3  
2                         452                              493         3  
3                         466                              662         3  
4                         358                              432         3  
..                        ...                              ...       ...  
795                       235                              238         3  
796                       553                              430         3  
797                        69                              171         3  
798                       218                              432         3  
799                       267                              139         3  

[800 rows x 7 columns]

# Optimization Solution WithOut Traffic Time (Return)

In [50]:
Sum_Cost_WithOut_Traffic_Return = 0
Avg_Cost_WithOut_Traffic_Return = 0

WithOutTrafic_Return = {}
WithOutTrafic_Return_optimal = []
WithOutTrafic_Return_frame_optimal = pd.DataFrame()

In [51]:
for key, group in itertools.groupby(all_data, key=lambda x: x["origin_id"]):

    data = {}
    data[key] = list(group)
    data = data[key]

    # Define the problem as a minimization problem
    problem = pulp.LpProblem("Ambulance Delivery Problem", pulp.LpMinimize)

    # Create decision variables
    decision_variables = []
    for i, row in enumerate(data):
        variable = pulp.LpVariable(f"X_{i}", lowBound=0, cat='Integer')
        decision_variables.append(variable)

    # Create objective function with decision variable
    total_cost = pulp.lpSum([(decision_variables[i] * (
                int(data[i]['Distance in meter']) + 10*int(data[i]['return duration in seconds']) * int(data[i]['severity'])))
                             for i in range(len(data))])
    problem += total_cost

    # Create constraints
    for i in range(len(data)):
        problem += (decision_variables[i] >= 0)
    problem += (pulp.lpSum([decision_variables[i] for i in range(len(data))]) == 1)

    # Solve problem with different algorithms
    solvers = [
        pulp.getSolver('PULP_CBC_CMD', timeLimit=1200),
        #pulp.GLPK_CMD(msg=True),
        #pulp.GUROBI(),
    ]

    for solver in solvers:
        problem.solve(solver)
        objective_value = pulp.value(problem.objective)
        Sum_Cost_WithOut_Traffic_Return +=objective_value
        WithOutTrafic_Return = {}
        WithOutTrafic_Return['cost_of_PULP_CBC_CMD'] = objective_value

        # Print optimal solution for every origin_id

        optimal_destinations = {}
        for i, row in enumerate(data):
            if decision_variables[i].value() > 0:
                origin_id = row['origin_id']
                destination_id = row['destination_id']
                if origin_id not in optimal_destinations or \
                        data[optimal_destinations[origin_id]]['return duration in seconds'] > row[
'return duration in seconds']:
                    optimal_destinations[origin_id] = i

        optimal_solutions = {}
        for origin_id in optimal_destinations:
            optimal_solutions[origin_id] = []
            for i in range(len(data)):
                row = data[i]
                if row['origin_id'] == origin_id and decision_variables[i].value() > 0:
                    optimal_solutions[origin_id].append((row['destination_id'], row['Distance in meter'],row['duration in seconds'],
                                                         row['return duration in seconds'], row['severity']))

        for origin_id in sorted(optimal_solutions.keys()):
            for x in optimal_solutions[origin_id]:
                   WithOutTrafic_Return['accident'] = origin_id
                   WithOutTrafic_Return['hospital'] = x[0]
                   WithOutTrafic_Return['distance'] = x[1]
                   WithOutTrafic_Return['duration without traffic'] = x[2]
                   WithOutTrafic_Return['return duration without traffic'] = x[3]
                   WithOutTrafic_Return['severity'] = x[4]
                   WithOutTrafic_Return_optimal.append(WithOutTrafic_Return)

C:\Users\wesam\anaconda3\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [52]:
print(WithOutTrafic_Return_optimal)

[{'cost_of_PULP_CBC_CMD': 26920.0, 'accident': '200701XH65182', 'hospital': 40095, 'distance': 5590, 'duration without traffic': 732, 'return duration without traffic': 711, 'severity': 3}, {'cost_of_PULP_CBC_CMD': 12353.0, 'accident': '200901QA10383', 'hospital': 40083, 'distance': 2033, 'duration without traffic': 337, 'return duration without traffic': 344, 'severity': 3}, {'cost_of_PULP_CBC_CMD': 17176.0, 'accident': '200501KF60564', 'hospital': 46586, 'distance': 2386, 'duration without traffic': 452, 'return duration without traffic': 493, 'severity': 3}, {'cost_of_PULP_CBC_CMD': 23651.0, 'accident': '201001RG40583', 'hospital': 462443, 'distance': 3791, 'duration without traffic': 466, 'return duration without traffic': 662, 'severity': 3}, {'cost_of_PULP_CBC_CMD': 11090.0, 'accident': '201201JC30566', 'hospital': 39964, 'distance': 2150, 'duration without traffic': 603, 'return duration without traffic': 298, 'severity': 3}, {'cost_of_PULP_CBC_CMD': 20314.0, 'accident': '201401

In [53]:
WithOutTrafic_Return_frame_optimal = pd.DataFrame.from_records(WithOutTrafic_Return_optimal)

In [54]:
WithOutTrafic_Return_frame_optimal.head(800)

cost_of_PULP_CBC_CMD       accident  hospital  distance  \
0                 26920.0  200701XH65182     40095      5590   
1                 12353.0  200901QA10383     40083      2033   
2                 17176.0  200501KF60564     46586      2386   
3                 23651.0  201001RG40583    462443      3791   
4                 11090.0  201201JC30566     39964      2150   
..                    ...            ...       ...       ...   
795                8249.0  201201QK50297     66513      1109   
796               15047.0  200601EO40267     66507      2147   
797                5846.0  201001RG40143   2704335       716   
798               15034.0  200601XD80655   2917654      2074   
799                4898.0  200801RY10207   1819387       728   

     duration without traffic  return duration without traffic  severity  
0                         732                              711         3  
1                         337                              344         3  
2                         452                              493         3  
3                         466                              662         3  
4                         603                              298         3  
..                        ...                              ...       ...  
795                       235                              238         3  
796                       553                              430         3  
797                        69                              171         3  
798                       218                              432         3  
799                       267                              139         3  

[800 rows x 7 columns]

#Result SUM

In [66]:
result_sum = {}

In [67]:
result_sum['name_of_optimizer'] = []
result_sum['Value'] = []

In [68]:
result_sum['name_of_optimizer'].append("Sum Cost With Traffic GO")
result_sum['Value'].append(Sum_Cost_With_Traffic_GO)
result_sum['name_of_optimizer'].append("Sum Cost WithOut Traffic GO")
result_sum['Value'].append(Sum_Cost_WithOut_Traffic_GO)

result_sum['name_of_optimizer'].append("Sum Cost With Traffic Return")
result_sum['Value'].append(Sum_Cost_With_Traffic_Return)
result_sum['name_of_optimizer'].append("Sum Cost WithOut Traffic Return")
result_sum['Value'].append(Sum_Cost_WithOut_Traffic_Return)

result_sum['name_of_optimizer'].append("Sum Cost With Traffic (same hospitl)")
result_sum['Value'].append(Sum_Cost_With_Traffic)
result_sum['name_of_optimizer'].append("Sum Cost WithOut Traffic (same hospitl)")
result_sum['Value'].append(Sum_Cost_WithOut_Traffic)

result_sum['name_of_optimizer'].append("Sum Cost With Traffic (not same hospitl)")
result_sum['Value'].append(Sum_Cost_With_Traffic_GO + Sum_Cost_With_Traffic_Return)
result_sum['name_of_optimizer'].append("Sum Cost WithOut Traffic (not same hospitl)")
result_sum['Value'].append(Sum_Cost_WithOut_Traffic_GO + Sum_Cost_WithOut_Traffic_Return)

In [69]:
df = pd.DataFrame(result_sum)
df

name_of_optimizer       Value
0                     Sum Cost With Traffic GO  11124259.0
1                  Sum Cost WithOut Traffic GO  11385344.0
2                 Sum Cost With Traffic Return  10962408.0
3              Sum Cost WithOut Traffic Return  11087340.0
4         Sum Cost With Traffic (same hospitl)  22352410.0
5      Sum Cost WithOut Traffic (same hospitl)  22736288.0
6     Sum Cost With Traffic (not same hospitl)  22086667.0
7  Sum Cost WithOut Traffic (not same hospitl)  22472684.0

# Result Average

In [70]:
result_avg = {}

In [71]:
result_avg['name_of_optimizer'] = []
result_avg['Value'] = []

In [72]:
result_avg['name_of_optimizer'].append("Avarege Cost With Traffic GO")
result_avg['Value'].append(Sum_Cost_With_Traffic_GO / len_accident)
result_avg['name_of_optimizer'].append("Avarege Cost WithOut Traffic GO")
result_avg['Value'].append(Sum_Cost_WithOut_Traffic_GO / len_accident)

result_avg['name_of_optimizer'].append("Avarege Cost With Traffic Return")
result_avg['Value'].append(Sum_Cost_With_Traffic_Return / len_accident)
result_avg['name_of_optimizer'].append("Avarege Cost WithOut Traffic Return")
result_avg['Value'].append(Sum_Cost_WithOut_Traffic_Return / len_accident)

result_avg['name_of_optimizer'].append("Avarege Cost With Traffic (same hospitl)")
result_avg['Value'].append(Sum_Cost_With_Traffic / len_accident)
result_avg['name_of_optimizer'].append("Avarege Cost WithOut Traffic (same hospitl)")
result_avg['Value'].append(Sum_Cost_WithOut_Traffic / len_accident)


result_avg['name_of_optimizer'].append("Avarege Cost With Traffic (not same hospitl)")
result_avg['Value'].append((Sum_Cost_With_Traffic_GO + Sum_Cost_With_Traffic_Return)/len_accident)
result_avg['name_of_optimizer'].append("Avarege Cost WithOut Traffic (not same hospitl)")
result_avg['Value'].append((Sum_Cost_WithOut_Traffic_GO + Sum_Cost_WithOut_Traffic_Return)/len_accident)

In [73]:
df = pd.DataFrame(result_avg)
df

name_of_optimizer        Value
0                     Avarege Cost With Traffic GO  13905.32375
1                  Avarege Cost WithOut Traffic GO  14231.68000
2                 Avarege Cost With Traffic Return  13703.01000
3              Avarege Cost WithOut Traffic Return  13859.17500
4         Avarege Cost With Traffic (same hospitl)  27940.51250
5      Avarege Cost WithOut Traffic (same hospitl)  28420.36000
6     Avarege Cost With Traffic (not same hospitl)  27608.33375
7  Avarege Cost WithOut Traffic (not same hospitl)  28090.85500